In [1]:

# =========================================== Refactored & Extended anomaly analysis pipeline =========================================== #

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import boxcox
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import (
    silhouette_score, confusion_matrix, classification_report,
    precision_recall_fscore_support
)
from sklearn.inspection import permutation_importance
from sklearn.decomposition import PCA
import time

sns.set(style="whitegrid")

# Like the name says, an heartbeat, to check if the system is alive
import threading, time

def heartbeat():
    while True:
        time.sleep(3600)  # 1 hour
        print("Running...")

thread = threading.Thread(target=heartbeat, daemon=True)
thread.start()

import json

In [2]:
# -------------------------
# 0. Config / Paths
# -------------------------
TRAIN_PATH = r'X:\Dissertacao\python_projects\dataset\ISCX-Bot-2014\ISCX_csv\Testing_file.csv'
TEST_PATH  = r'X:\Dissertacao\python_projects\dataset\ISCX-Bot-2014\ISCX_csv\Training_file.csv'
LABEL_COL = "Label"   # set to your label column name if exists, otherwise leave as-is
SAMPLE_FOR_SIL = 40000
SIL_BATCH = 40000
LOF_BATCH = 200_000
RANDOM_STATE = 42

In [3]:
# -------------------------
# 1. Load data
# -------------------------
def load_datasets(train_path, test_path):
    df_tr = pd.read_csv(train_path, encoding='ISO-8859-1')
    df_te = pd.read_csv(test_path, encoding='ISO-8859-1')
    return df_tr, df_te

df_train, df_test = load_datasets(TRAIN_PATH, TEST_PATH)

In [4]:
# -------------------------
# 2. Basic cleaning
# -------------------------
# Fill Info NaNs and drop rows missing Source/Destination
df_train['Info'].fillna("Unknown", inplace=True)
df_test['Info'].fillna("Unknown", inplace=True)
df_train.dropna(subset=['Source','Destination'], inplace=True)
df_test.dropna(subset=['Source','Destination'], inplace=True)

# Remove rows with negative Time (if that's desired)
df_train = df_train[df_train['Time'] >= 0].copy()
df_test  = df_test[df_test['Time'] >= 0].copy()

C:\Users\alexs\AppData\Local\Temp\ipykernel_15036\1138178066.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_train['Info'].fillna("Unknown", inplace=True)
C:\Users\alexs\AppData\Local\Temp\ipykernel_15036\1138178066.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exa

In [5]:
# ==================== Build global mappings ==================== #
def build_mapping(train_col, test_col, name):
    """Create a dictionary mapping for train+test, save to JSON, return mapping."""
    all_values = pd.concat([train_col, test_col]).unique()
    mapping = {val: idx for idx, val in enumerate(all_values)}

    # optional: save for reproducibility
    with open(f"{name}_mapping.json", "w") as f:
        json.dump(mapping, f)

    return mapping

# Protocol
protocol_mapping = build_mapping(df_train['Protocol'], df_test['Protocol'], "protocol")
df_train['Protocol_enc'] = df_train['Protocol'].map(protocol_mapping).fillna(-1).astype(int)
df_test['Protocol_enc']  = df_test['Protocol'].map(protocol_mapping).fillna(-1).astype(int)

# Source
source_mapping = build_mapping(df_train['Source'], df_test['Source'], "source")
df_train['Source_enc'] = df_train['Source'].map(source_mapping).fillna(-1).astype(int)
df_test['Source_enc']  = df_test['Source'].map(source_mapping).fillna(-1).astype(int)

# Destination
dest_mapping = build_mapping(df_train['Destination'], df_test['Destination'], "destination")
df_train['Destination_enc'] = df_train['Destination'].map(dest_mapping).fillna(-1).astype(int)
df_test['Destination_enc']  = df_test['Destination'].map(dest_mapping).fillna(-1).astype(int)

# ==================== Drop originals (optional) ==================== #
# Keep only the encoded versions for modeling
df_train.drop(['Protocol', 'Source', 'Destination'], axis=1, inplace=True)
df_test.drop(['Protocol', 'Source', 'Destination'], axis=1, inplace=True)

In [6]:
# -------------------------
# 4. Feature engineering (as in your code, with small safety tweaks)
# -------------------------
def build_features(df):
    # Time diff per Source
    df['Time_Diff'] = df.groupby("Source")['Time'].diff()
    # If first diff is NaN, fill with group's median diff (safer than mean)
    df['Time_Diff'] = df['Time_Diff'].fillna(df.groupby('Source')['Time'].transform('median'))
    df['Time_Diff'] = df['Time_Diff'].fillna(0.0)
    # Packet rate: robust clip then packets / time span (clip prevents tiny denominators)
    def packet_rate(series):
        if len(series) < 2:
            return 0.0
        q1, q3 = np.percentile(series, [25, 75])
        iqr = max(q3 - q1, 1e-9)
        clipped = np.clip(series, q1 - 1.5 * iqr, q3 + 1.5 * iqr)
        denom = clipped.max() - clipped.min()
        denom = denom if denom > 1e-6 else 1e-6
        return len(clipped) / denom
    df['Packet_Rate'] = df.groupby('Source')['Time'].transform(packet_rate)
    # Inter-arrival (rolling mean)
    df['Inter_Arrival_Time'] = df.groupby('Source')['Time_Diff'].transform(lambda x: x.rolling(10, min_periods=1).mean())
    df['Inter_Arrival_Time'] = df['Inter_Arrival_Time'].clip(lower=1e-6)
    df['Burst_Rate'] = np.where(df['Inter_Arrival_Time'] > 1e-6, 1.0 / df['Inter_Arrival_Time'], 0.0)
    # Transforms
    df['Log_IATime'] = np.log1p(df['Inter_Arrival_Time'])
    df['Log_BRate']  = np.log1p(df['Burst_Rate'].clip(lower=0))
    # BoxCox requires positive. add small epsilon
    df['BoxCox_Length'] = df['Length'] + 1e-3
    df['BoxCox_Length'], _ = boxcox(df['BoxCox_Length'])
    df['BoxCox_PRate'] = df['Packet_Rate'] + 1e-6
    # BoxCox may fail if constant; guard with log1p fallback
    try:
        df['BoxCox_PRate'], _ = boxcox(df['BoxCox_PRate'])
    except Exception:
        df['BoxCox_PRate'] = np.log1p(df['Packet_Rate'])
    return df

df_train = build_features(df_train)
df_test  = build_features(df_test)

KeyError: 'Source'

In [ ]:
# -------------------------
# 5. Select features & scale (fit scaler on train, apply to test)
# -------------------------
NUM_FEATURES = ["Time_Diff", "Log_IATime", "Log_BRate", "BoxCox_Length", "BoxCox_PRate"]
scaler = MinMaxScaler()
scaler.fit(df_train[NUM_FEATURES])
df_train_scaled = pd.DataFrame(scaler.transform(df_train[NUM_FEATURES]), columns=NUM_FEATURES)
df_test_scaled  = pd.DataFrame(scaler.transform(df_test[NUM_FEATURES]), columns=NUM_FEATURES)

# Small de-duplication + tiny noise to avoid LOF duplicate issues
df_train_scaled = df_train_scaled.drop_duplicates().reset_index(drop=True)
df_test_scaled  = df_test_scaled.drop_duplicates().reset_index(drop=True)
df_train_scaled += np.random.normal(0, 1e-6, df_train_scaled.shape).astype("float32")
df_test_scaled  += np.random.normal(0, 1e-6, df_test_scaled.shape).astype("float32")

# Reduce precision to save memory
df_train_scaled = df_train_scaled.astype("float32")
df_test_scaled  = df_test_scaled.astype("float32")

# We'll keep copies of the scaled frames with indexes aligned to original for backreference:
df_train_scaled_full = df_train_scaled.copy()
df_test_scaled_full  = df_test_scaled.copy()

In [ ]:
# -------------------------
# 6. Train Isolation Forest
# -------------------------
iso_forest = IsolationForest(n_estimators=300, contamination="auto", max_samples=100000, random_state=RANDOM_STATE, verbose=0)
t0 = time.time()
iso_forest.fit(df_train_scaled_full)
t1 = time.time()
print(f"IsolationForest trained in {t1-t0:.1f}s")

df_train_scaled_full["Anomaly_IForest"] = iso_forest.predict(df_train_scaled_full)
df_test_scaled_full  = df_test_scaled_full if 'df_test_scaled_full' in globals() else df_test_scaled.copy()
df_test_scaled_full["Anomaly_IForest"] = iso_forest.predict(df_test_scaled)

# Convert labels to -1/1 convention (already so)
print("IForest train anomaly %:", (df_train_scaled_full["Anomaly_IForest"]==-1).mean()*100)
print("IForest test anomaly %: ", (df_test_scaled_full["Anomaly_IForest"]==-1).mean()*100)

In [ ]:
# -------------------------
# 7. Batched LOF predictions (memory-friendly)
# -------------------------
lof = LocalOutlierFactor(n_neighbors=30, algorithm="kd_tree", leaf_size=40, contamination="auto", metric="euclidean", novelty=False, n_jobs=-1)

def lof_predict_batched(X, batch_size=LOF_BATCH, lof_model=None):
    """Return concatenated lof predictions computed per-batch.
       NOTE: LOF.fit_predict computes scores per-batch independently (no incremental LOF)."""
    if lof_model is None:
        lof_model = LocalOutlierFactor(n_neighbors=30, metric="euclidean", contamination="auto")
    n = len(X)
    y = np.zeros(n, dtype=int)
    for i in range(0, n, batch_size):
        batch = X.iloc[i:i+batch_size]
        # fit_predict returns -1 for outlier, 1 for inlier
        y[i:i+len(batch)] = lof_model.fit_predict(batch)
    return y

t0 = time.time()
y_lof_train = lof_predict_batched(df_train_scaled_full, batch_size=LOF_BATCH, lof_model=lof)
t1 = time.time()
print(f"LOF (train) batched predictions in {t1-t0:.1f}s")

y_lof_test = lof_predict_batched(df_test_scaled, batch_size=LOF_BATCH, lof_model=lof)
df_train_scaled_full["Anomaly_LOF"] = y_lof_train
df_test_scaled_full["Anomaly_LOF"] = y_lof_test

print("LOF train anomaly %:", (df_train_scaled_full["Anomaly_LOF"]==-1).mean()*100)
print("LOF test anomaly %: ", (df_test_scaled_full["Anomaly_LOF"]==-1).mean()*100)

In [ ]:
# -------------------------
# 8. Agreement & simple stats
# -------------------------
agreement_train = (df_train_scaled_full["Anomaly_IForest"] == df_train_scaled_full["Anomaly_LOF"]).mean()
print(f"Model agreement on train: {agreement_train:.2%}")

In [ ]:
# -------------------------
# 9. Silhouette (batched sampling)
# -------------------------
def batched_silhouette(X, labels_col, batch_size=SAMPLE_FOR_SIL, metric='euclidean', n_batches=5):
    scores = []
    n = len(X)
    # sample n_batches different random batches
    rng = np.random.RandomState(RANDOM_STATE)
    for i in range(n_batches):
        sample_idx = rng.choice(n, min(batch_size, n), replace=False)
        batch = X.iloc[sample_idx]
        labels = batch[labels_col].values
        # silhouette requires at least 2 clusters with more than 1 sample each
        if len(np.unique(labels)) < 2 or min(np.bincount((labels==-1).astype(int))) < 2:
            scores.append(np.nan)
            continue
        try:
            sc = silhouette_score(batch[NUM_FEATURES], labels, metric=metric)
            scores.append(sc)
        except Exception:
            scores.append(np.nan)
    return np.array(scores)

sil_if = batched_silhouette(df_train_scaled_full.assign(**{ 'labels_if' : df_train_scaled_full["Anomaly_IForest"]}), 'Anomaly_IForest', batch_size=SAMPLE_FOR_SIL, n_batches=5, metric='euclidean')
sil_lof= batched_silhouette(df_train_scaled_full.assign(**{ 'labels_lof': df_train_scaled_full["Anomaly_LOF"]}), 'Anomaly_LOF', batch_size=SAMPLE_FOR_SIL, n_batches=5, metric='euclidean')

print("Silhouette IForest (mean of batches):", np.nanmean(sil_if))
print("Silhouette LOF     (mean of batches):", np.nanmean(sil_lof))

In [ ]:
# -------------------------
# 10. Feature-level summaries & plots (KDEs etc)
# -------------------------
def feature_summary_and_plots(df_scaled_full, original_df, model_cols, features):
    """For each model in model_cols, produce feature summaries and KDEs (side-by-side)."""
    summaries = {}
    for model_col in model_cols:
        summary = {}
        summary['anomaly_rate'] = (df_scaled_full[model_col] == -1).mean() * 100
        feat_stats = {}
        for f in features:
            # map back scaled -> original feature values roughly using the scaler inverse
            # but we also can read original_df to get real values aligned: careful w/ dedup.
            # For simplicity we compute stats from original_df filtered by mask aligned via index if same size.
            mask = (df_scaled_full[model_col] == -1)
            feat_stats[f] = {
                'mean_normal': original_df.loc[~mask, f].mean() if f in original_df.columns else np.nan,
                'mean_anomaly': original_df.loc[mask, f].mean() if f in original_df.columns else np.nan,
                'median_normal': original_df.loc[~mask, f].median() if f in original_df.columns else np.nan,
                'median_anomaly': original_df.loc[mask, f].median() if f in original_df.columns else np.nan,
                'anomaly_count': mask.sum()
            }
        summary['feature_stats'] = feat_stats
        summaries[model_col] = summary

    # Plots: KDE side-by-side for each feature
    for f in features:
        fig, axes = plt.subplots(1, len(model_cols), figsize=(5 * len(model_cols), 4))
        if len(model_cols) == 1:
            axes = [axes]
        for ax, model_col in zip(axes, model_cols):
            mask_an = df_scaled_full[model_col] == -1
            mask_no = df_scaled_full[model_col] == 1
            # attempt to plot using original_df if possible to keep units
            data_normal = original_df.loc[mask_no, f] if f in original_df.columns else df_scaled_full.loc[mask_no, f]
            data_anom   = original_df.loc[mask_an, f] if f in original_df.columns else df_scaled_full.loc[mask_an, f]
            if data_normal.dropna().shape[0] > 1:
                sns.kdeplot(data_normal, ax=ax, label='Normal', fill=True, color='blue')
            if data_anom.dropna().shape[0] > 1:
                sns.kdeplot(data_anom, ax=ax, label='Anomaly', fill=True, color='red')
            ax.set_title(f"{model_col} - {f}")
            ax.legend()
        plt.tight_layout()
        plt.show()
    return summaries

# Build summaries for train (use original df_train for real feature units)
model_cols = ["Anomaly_IForest", "Anomaly_LOF"]
train_summaries = feature_summary_and_plots(df_train_scaled_full, df_train.reset_index(drop=True), model_cols, ["BoxCox_Length","Log_BRate","Log_IATime"])
print("Train summaries done.")

In [ ]:
# -------------------------
# 11. If labels exist -> compute confusion & per-feature TP/FP/FN/TN
# -------------------------
def compute_confusion_and_feature_errors(original_df, df_scaled_full, model_col, label_col=LABEL_COL, features=NUM_FEATURES):
    """If ground truth exists, compute confusion and per-feature breakdown."""
    if label_col not in original_df.columns:
        return None
    # Normalize label: try to map -1/1 or 0/1 to 1=anomaly
    y_true = original_df[label_col].copy().astype(int)
    # If labels are 1 (normal) 0 (anomaly) detect and transform: assume anomalies are 1 when values {-1,1} present
    if set(y_true.unique()) <= {0,1}:  # 0/1, assume 1=anomaly? ambiguous -> attempt to detect majority
        # we assume 1 means anomaly only if anomalies are minority; otherwise try -1/1
        pass
    # We'll make y_true_binary such that 1=anomaly, 0=normal
    if set(y_true.unique()) == {-1,1}:
        y_true_bin = (y_true == -1).astype(int)
    elif set(y_true.unique()) <= {0,1}:
        # assume 1 == anomaly if anomaly labels are rare; otherwise ask user — but here we map 1 -> anomaly
        y_true_bin = y_true.copy()
    else:
        # unknown encoding
        y_true_bin = y_true.copy()

    y_pred = (df_scaled_full[model_col] == -1).astype(int)
    cm = confusion_matrix(y_true_bin[:len(y_pred)], y_pred)
    report = classification_report(y_true_bin[:len(y_pred)], y_pred, digits=4)
    # per-feature error rates: compute FP rate among samples where feature in top quantile, etc.
    per_feature = {}
    for feat in features:
        if feat in original_df.columns:
            q75 = original_df[feat].quantile(0.75)
            high_mask = original_df[feat] >= q75
            # in this high-value subgroup: compute precision/recall
            if high_mask.sum() > 0:
                y_t = y_true_bin[high_mask]
                y_p = y_pred[high_mask][:len(y_t)]
                prec, rec, f1, _ = precision_recall_fscore_support(y_t, y_p, average='binary', zero_division=0)
            else:
                prec, rec, f1 = np.nan, np.nan, np.nan
            per_feature[feat] = {'q75_count': int(high_mask.sum()), 'precision_high': prec, 'recall_high': rec, 'f1_high': f1}
    return {'confusion_matrix': cm, 'classification_report': report, 'per_feature_high': per_feature}

# check for train labels
train_conf = compute_confusion_and_feature_errors(df_train.reset_index(drop=True), df_train_scaled_full, 'Anomaly_IForest', LABEL_COL, features=["BoxCox_Length","Log_BRate","Log_IATime"])
if train_conf:
    print("Confusion and feature-level errors (train):")
    print(train_conf['classification_report'])

In [ ]:
# -------------------------
# 12. Permutation importance for feature relevance (IsolationForest)
# -------------------------
# Permutation importance needs a predict_proba-like continuous score. For IsolationForest we can use decision_function
try:
    iso_scores = iso_forest.decision_function(df_train_scaled_full[NUM_FEATURES])
    r = permutation_importance(iso_forest, df_train_scaled_full[NUM_FEATURES], iso_scores, n_repeats=10, random_state=RANDOM_STATE, n_jobs=-1)
    imp_df = pd.DataFrame({'feature': NUM_FEATURES, 'importance_mean': r.importances_mean, 'importance_std': r.importances_std})
    imp_df.sort_values('importance_mean', ascending=False, inplace=True)
    print("Permutation importances (IForest):")
    print(imp_df)
except Exception as e:
    print("Permutation importance failed:", e)

In [ ]:
# -------------------------
# 13. Dotted summary plot: per-feature comparison (anomaly rates and (if available) proxy precision)
# -------------------------
def dotted_summary_plot(df_scaled_full, original_df, model_cols, features):
    recs = []
    for model_col in model_cols:
        for f in features:
            mask_an = df_scaled_full[model_col] == -1
            # anomaly rate in top quantile for this feature
            if f in original_df.columns:
                q75 = original_df[f].quantile(0.75)
                top = original_df[f] >= q75
                # proportion of anomalies within top quantile:
                prop_in_top = (mask_an[top.values]).sum() / max(1, top.sum())
            else:
                prop_in_top = np.nan
            recs.append({'model': model_col, 'feature': f, 'anomaly_rate': (mask_an.mean()*100),'prop_top_q75': prop_in_top})
    df_plot = pd.DataFrame(recs)
    plt.figure(figsize=(8,5))
    for model, g in df_plot.groupby('model'):
        plt.scatter(g['feature'], g['anomaly_rate'], s=100, label=model)
    plt.ylabel('Anomaly Rate (%)')
    plt.title('Per-feature anomaly rates (by model)')
    plt.legend()
    plt.show()
    return df_plot

df_plot = dotted_summary_plot(df_train_scaled_full, df_train.reset_index(drop=True), model_cols, ["BoxCox_Length","Log_BRate","Log_IATime"])
print("Dotted summary table:")
print(df_plot)

# -------------------------
# End - prints and small guidance
# -------------------------
print("Done. Key artifacts:")
print(f" - df_train_scaled_full, df_test_scaled_full contain scaled features and model labels.")
print(" - train_summaries contains per-model feature stats returned earlier.")
print(" - df_plot contains the dotted-summary aggregated table.")
print("Next: inspect train_summaries and df_plot, examine KDEs and permutation importances.")